In [4]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import azureml
from azureml.core import Workspace, Run
from azureml.core.model import Model

In [5]:
# Connect to Azure Machine Learning (aml) Workspace
# set aml workspace parameters here. 
subscription_id = "c0e490ed-dbf1-4955-9ea0-597b55b9762a"
resource_group = "test"
workspace_name = "asl-model-workspace" # note that this is an Azure ML Workspace (had to create it in the actual service, NOT in Databricks)
workspace_region = "eastus"


# set container and account name for data sources
container_name = "asl-model-deployment-container"
account_name = "aslmodeldeployment"

ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)

In [ ]:
# # copy the model to local/current working directory for deployment
# model_name = "saved_model.pb"
# deploy_folder = os.getcwd() # get current working directory
# dbutils.fs.cp('dbfs:/mnt/aslmodeldeployment/asl-model-deployment-container/' + model_name, "file:" + deploy_folder + "/" + model_name, True)

In [ ]:
# Register the model for deployment
# register the model 
model_name = 'mnist-asl-cnn-model'
mymodel = Model.register(model_name = model_name, model_path = '/dbfs/mnt/aslmodeldeployment/asl-model-deployment-container/saved_model.pb', description = "Trained ASL CNN model", workspace = ws )

In [ ]:
!ls /dbfs/mnt/aslmodeldeployment/asl-model-deployment-container/

In [ ]:
# Write a scoring file that will be used when the web service is called
score = """
import json
import numpy as np
import os
import pickle
import joblib
from sklearn.linear_model import LogisticRegression

from azureml.core.model import Model

def init():    
    global model
    # retreive the path to the model file using the model name
    model_path = Model.get_model_path('{model_name}')
    #model_path = '/dbfs/mnt/aslmodeldeployment/asl-model-deployment-container/'
    model = joblib.load(model_path)
    
    
def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    
    # make prediction
    y_hat = model.predict(data)
    
    # you can return any data type as long as it is JSON-serializable
    return y_hat.tolist()
    
""".format(model_name=model_name)

exec(score)

with open("score.py", "w") as file:
    file.write(score)

In [ ]:
# Write a myenv.yml file
myenv = """

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.8 and later.
- python=3.8.13

- pip:
  - azureml-defaults
  - pyopenssl
- scikit-learn
channels:
- anaconda
- conda-forge
    
"""#.format(model_name=model_name)

#exec(myenv)

with open("myenv.yml", "w") as file:
    file.write(myenv)

In [ ]:
# Create config file for deployment / Create a dependencies file
# from azureml.core.conda_dependencies import CondaDependencies 

# myenv = CondaDependencies.create(conda_packages=['scikit-learn']) #showing how to add libs as an eg. - not needed for this model.

# with open("myenv.yml","w") as f:
#     f.write(myenv.serialize_to_string())

In [ ]:
# Configure an Azure Container Instance to deploy to. / ACI Configuration
# This will be deployed to our Azure Machine Learning Service Workspace (asl-model-workspace)
from azureml.core.webservice import AciWebservice, Webservice

myaci_config = AciWebservice.deploy_configuration(cpu_cores=1, 
            memory_gb=1, 
            tags={"data": "MNIST",  "method" : "CNN"}, 
            description='Predict ASL (MNIST) letters with a Keras CNN')

In [ ]:
# Configure image before deploying model as a web service
# deploy to aci
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                runtime="python", 
                                                conda_file="myenv.yml")

In [ ]:
# Deploy the model
service = Webservice.deploy_from_model(workspace=ws,
                                    name='mnist-asl-cnn-model',
                                    deployment_config=myaci_config,
                                    models=[mymodel],
                                    image_config=image_config)

service.wait_for_deployment(show_output=True)

In [ ]:
# Get service logs (good for when deployment fails)
# if you already have the service object handy
print(service.get_logs())

# if you know the service name
#print(ws.webservices()['mnist-asl-cnn-model'].get_logs())